In [1]:
print("Starting Finetuning...")

Starting Finetuning...


In [2]:
import kaggle_secrets
import os
import wandb

# Fetch API key from Kaggle Secrets
WANDB_API_KEY = kaggle_secrets.UserSecretsClient().get_secret("WANDB_API_KEY")

# Set as environment variable
os.environ["WANDB_API_KEY"] = WANDB_API_KEY

# Login to W&B
wandb.login(key=WANDB_API_KEY)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kiranchw000 (imnskc) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
## install requirements
# !bash /kaggle/working/TMLC/week2_unsloth_dpo_qlora/requirements.sh

In [4]:
%%capture
!pip install pip3-autoremove
!pip install datasets trl wandb
!pip-autoremove torch torchvision torchaudio -y
!pip install "torch==2.4.0" "xformers==0.0.27.post2" triton torchvision torchaudio
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

In [5]:
import shutil
import os

# # Example: remove a directory and all its contents
dir_path = "/kaggle/working"

# removing current directories to run this version
!rm -rf /kaggle/working/*

print(os.listdir(dir_path))

['.virtual_documents']


In [6]:
!git clone https://github.com/kishkath/TMLC.git

Cloning into 'TMLC'...
remote: Enumerating objects: 637, done.
remote: Counting objects: 100% (171/171), done.
remote: Compressing objects: 100% (171/171), done.
remote: Total 637 (delta 111), reused 0 (delta 0), pack-reused 466 (from 2)
Receiving objects: 100% (637/637), 391.62 KiB | 5.09 MiB/s, done.
Resolving deltas: 100% (372/372), done.


In [7]:
# !pip install unsloth_zoo

In [8]:
!python /kaggle/working/TMLC/week2_unsloth_dpo_qlora/main.py

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
2025-10-05 03:55:05.382617: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759636505.783971     191 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759636505.889085     191 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
🦥 Unsloth Zoo will now patch everything to make training faster!
2025-10-05 03:55:38,138 | INFO | Preparing dataset...
Generating train split: 6000 examples [00:00, 33793.74 examples/s]
Generating val split: 1000 examples [00:00, 66216.79 examples/s]
Filter: 100%|████████████████████| 1000/1000 [00:00<00:00, 134119.02 examples/s]
2025-10-05 03:55:39,094 | INFO |

In [9]:
!python /kaggle/working/TMLC/week2_unsloth_dpo_qlora/inference.py

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
2025-10-05 05:47:27.093971: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759643247.117748     631 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759643247.125177     631 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
🦥 Unsloth Zoo will now patch everything to make training faster!
2025-10-05 05:47:38,570 | INFO | 🔍 Loading fine-tuned model from 'outputs'
2025-10-05 05:47:38,570 | INFO | Loading fine-tuned model from: outputs
==((====))==  Unsloth 2025.9.11: Fast Qwen3 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux

In [11]:
import os
os.chdir("/kaggle/working/TMLC/week2_unsloth_dpo_qlora/")
from configurations.config import INFERENCE_MODEL_PATH, INFERENCE_MAX_SEQ_LENGTH, INFERENCE_LOAD_IN_4BIT, logger
from core.model_loader import load_finetuned_model
from core.inference_utils import predict

if __name__ == "__main__":
    logger.info(f"🔍 Loading fine-tuned model from '{INFERENCE_MODEL_PATH}'")
    try:
        INFERENCE_MODEL_PATH = "/kaggle/working/outputs/"
        model, tokenizer = load_finetuned_model(
            INFERENCE_MODEL_PATH, INFERENCE_MAX_SEQ_LENGTH, INFERENCE_LOAD_IN_4BIT
        )
        logger.info(f"✅ Fine-tuned model loaded successfully from '{INFERENCE_MODEL_PATH}'")
    except Exception as e:
        logger.error(f"❌ Failed to load model: {e}")
        raise

    input_prompt = "What are the most effective leg exercies. Just give me names of 3 exercises with no explanation."
    logger.info(f"💬 Running inference for prompt: {input_prompt}")

    try:
        response = predict(model, tokenizer, input_prompt)
        logger.info(f"✅ Prediction completed")
        print("\n💡 Prediction:\n", response)
    except Exception as e:
        logger.error(f"❌ Inference failed: {e}")

2025-10-05 05:54:19,359 | INFO | 🔍 Loading fine-tuned model from 'outputs'
2025-10-05 05:54:19,360 | INFO | Loading fine-tuned model from: /kaggle/working/outputs/


==((====))==  Unsloth 2025.9.11: Fast Qwen3 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.4.0+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.0.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.9.11 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.
2025-10-05 05:54:25,717 | INFO | ✅ Fine-tuned model loaded for inference
2025-10-05 05:54:25,717 | INFO | ✅ Fine-tuned model loaded successfully from '/kaggle/working/outputs/'
2025-10-05 05:54:25,718 | INFO | 💬 Running inference for prompt: What are the most effective leg exercies. Just give me names of 3 exercises with no explanation.
2025-10-05 05:54:25,718 | INFO | 💬 Starting prediction...
2025-10-05 05:54:25,719 | INFO | Input prompt: What are the most effective leg exercies. Just give me names of 3 exercises with no explanation.
2025-10-05 05:54:25,720 | INFO | System prompt: You are a helpful AI fitness assistant for gym-goers and vegetarians.
2025-10-05 05:54:25,720 | INFO | 🔧 Applying chat template to messages...
2025-10-05 05:54:25,721 | INFO | Formatted text: <|im_start|>system
You are a helpful AI fitness assistant for gym-goers and vegetarians.<|im_end|>
<|im_start|>user
What are the most e


💡 Prediction:
 <think>
Okay, the user is asking for the most effective leg exercises. They want three exercises without any explanation. Let me think. First, I need to recall common exercises for the legs. The user is likely a gym-goer, so they might be looking for something quick and efficient. 

I should consider compound exercises that target multiple muscle groups. For legs, squats, push-ups, and planks come to mind. But the user wants three with no explanation. So, I can list them but mention that each has a brief explanation. Wait, the user said "no explanation" so I can't add that. 

Let me check if I'm mixing up different muscles. Squats are good for strength and stability, push-ups for upper body, and planks for core. But the user wants three leg exercises. So, maybe: 1. Squat, 2. Deadlift, 3. Leg Press. But need to confirm if these are commonly used. 

Alternatively, maybe planks, push-ups, and squats. But the user said no explanation. So I'll go with three exercises, each w

In [12]:
import os
os.chdir("/kaggle/working/TMLC/week2_unsloth_dpo_qlora/")
from configurations.config import INFERENCE_MODEL_PATH, INFERENCE_MAX_SEQ_LENGTH, INFERENCE_LOAD_IN_4BIT, logger
from core.model_loader import load_finetuned_model
from core.inference_utils import predict

if __name__ == "__main__":
    logger.info(f"🔍 Loading fine-tuned model from '{INFERENCE_MODEL_PATH}'")
    try:
        INFERENCE_MODEL_PATH = "/kaggle/working/outputs/"
        model, tokenizer = load_finetuned_model(
            INFERENCE_MODEL_PATH, INFERENCE_MAX_SEQ_LENGTH, INFERENCE_LOAD_IN_4BIT
        )
        logger.info(f"✅ Fine-tuned model loaded successfully from '{INFERENCE_MODEL_PATH}'")
    except Exception as e:
        logger.error(f"❌ Failed to load model: {e}")
        raise

    input_prompt = "Is whey protein good enough to consume. Just say yes or no?"
    logger.info(f"💬 Running inference for prompt: {input_prompt}")

    try:
        response = predict(model, tokenizer, input_prompt)
        logger.info(f"✅ Prediction completed")
        print("\n💡 Prediction:\n", response)
    except Exception as e:
        logger.error(f"❌ Inference failed: {e}")

2025-10-05 05:56:23,147 | INFO | 🔍 Loading fine-tuned model from 'outputs'
2025-10-05 05:56:23,149 | INFO | Loading fine-tuned model from: /kaggle/working/outputs/


==((====))==  Unsloth 2025.9.11: Fast Qwen3 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.4.0+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.0.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


2025-10-05 05:56:29,190 | INFO | ✅ Fine-tuned model loaded for inference
2025-10-05 05:56:29,191 | INFO | ✅ Fine-tuned model loaded successfully from '/kaggle/working/outputs/'
2025-10-05 05:56:29,191 | INFO | 💬 Running inference for prompt: Is whey protein good enough to consume. Just say yes or no?
2025-10-05 05:56:29,192 | INFO | 💬 Starting prediction...
2025-10-05 05:56:29,193 | INFO | Input prompt: Is whey protein good enough to consume. Just say yes or no?
2025-10-05 05:56:29,193 | INFO | System prompt: You are a helpful AI fitness assistant for gym-goers and vegetarians.
2025-10-05 05:56:29,194 | INFO | 🔧 Applying chat template to messages...
2025-10-05 05:56:29,195 | INFO | Formatted text: <|im_start|>system
You are a helpful AI fitness assistant for gym-goers and vegetarians.<|im_end|>
<|im_start|>user
Is whey protein good enough to consume. Just say yes or no?<|im_end|>
<|im_start|>assistant

2025-10-05 05:56:29,197 | INFO | Using device: cuda
2025-10-05 05:56:29,219 | INFO |


💡 Prediction:
 <think>
Okay, the user is asking if whey protein is good enough to consume. They want a simple "yes" or "no" answer. Let me break this down.

First, I should consider the common perception. Many people believe that whey is the best, but I need to be honest. There are other factors to consider. Let me check the information I have. 

Whey protein is a good source of high-quality protein, containing all the amino acids in the right ratios. It's also rich in essential fats and is easy to digest. However, there are other benefits like muscle synthesis and a lean body build. 

But the user might be looking for a quick answer without going into details. So I should state "yes" but mention that it's not the only option. Wait, the user just wants "yes" or "no". Oh, right, the answer should be "yes" and then explain. But the original question is "Is whey protein good enough to consume. Just say yes or no?" So the answer is "yes". But then I need to add a bit more context. Wait, t

In [19]:
!zip -r /kaggle/working/unsloth_dpo_qlora.zip /kaggle/working/huggingface_tokenizers_cache /kaggle/working/logs /kaggle/working/outputs

  adding: kaggle/working/huggingface_tokenizers_cache/ (stored 0%)
  adding: kaggle/working/huggingface_tokenizers_cache/.locks/ (stored 0%)
  adding: kaggle/working/huggingface_tokenizers_cache/.locks/models--unsloth--Qwen3-0.6B-unsloth-bnb-4bit/ (stored 0%)
  adding: kaggle/working/huggingface_tokenizers_cache/.locks/models--unsloth--Qwen3-0.6B-unsloth-bnb-4bit/370b963bb5f261aa995ffeeb88d83a1d71dbb084.lock (stored 0%)
  adding: kaggle/working/huggingface_tokenizers_cache/.locks/models--unsloth--Qwen3-0.6B-unsloth-bnb-4bit/4783fe10ac3adce15ac8f358ef5462739852c569.lock (stored 0%)
  adding: kaggle/working/huggingface_tokenizers_cache/.locks/models--unsloth--Qwen3-0.6B-unsloth-bnb-4bit/b54f9135e44c1e81047e8d05cb027af8bc039eed.lock (stored 0%)
  adding: kaggle/working/huggingface_tokenizers_cache/.locks/models--unsloth--Qwen3-0.6B-unsloth-bnb-4bit/9b8043f10c758210957b050c77f14d6282f33a52.lock (stored 0%)
  adding: kaggle/working/huggingface_tokenizers_cache/.locks/models--unsloth--Qwen3-

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 61%)
  adding: kaggle/working/huggingface_tokenizers_cache/models--unsloth--Qwen3-0.6B-unsloth-bnb-4bit/snapshots/e86f5289a2ca40ab7114071959dc2079cf97cb3c/chat_template.jinja (deflated 77%)
  adding: kaggle/working/huggingface_tokenizers_cache/models--unsloth--Qwen3-0.6B-unsloth-bnb-4bit/snapshots/e86f5289a2ca40ab7114071959dc2079cf97cb3c/merges.txt (deflated 57%)
  adding: kaggle/working/huggingface_tokenizers_cache/models--unsloth--Qwen3-0.6B-unsloth-bnb-4bit/snapshots/e86f5289a2ca40ab7114071959dc2079cf97cb3c/added_tokens.json (deflated 68%)
  adding: kaggle/working/huggingface_tokenizers_cache/models--unsloth--Qwen3-0.6B-unsloth-bnb-4bit/snapshots/e86f5289a2ca40ab7114071959dc2079cf97cb3c/tokenizer.json (deflated 81%)
  adding: kaggle/working/huggingface_tokenizers_cache/models--unsloth--Qwen3-0.6B-unsloth-bnb-4bit/snapshots/e86f5289a2ca40ab7114071959dc2079cf97cb3c/tokenizer_config.json (deflated 84%)
  adding: kaggle/working/huggingface_tokenizers_cache/models--unsloth--Qw

In [20]:
import os
os.listdir("/kaggle/working/")

['v0.1_unsloth_dpo_qlora.zip',
 'unsloth_dpo_qlora.zip',
 'wandb',
 'TMLC',
 '.virtual_documents',
 'outputs',
 'huggingface_tokenizers_cache',
 'logs',
 'unsloth_compiled_cache']

In [21]:
from IPython.display import FileLink

# Create a clickable download link
FileLink("/kaggle/working/unsloth_dpo_qlora.zip")

/kaggle/working/unsloth_dpo_qlora.zip

In [22]:
from IPython.display import HTML

HTML('<a href="/kaggle/working/unsloth_dpo_qlora.zip" download>Download v0.1_unsloth_dpo_qlora.zip</a>')


In [23]:
import os

os.remove("/kaggle/working/v0.1_unsloth_dpo_qlora.zip")